# NodeJS server

Tento notebook slouží pro experimentální online spouštění single page html s podporou knihovny React.
Implementovaný server (express.js) vezme prvky - buňky z odkazovaného notebooku, filtruje buňky s kódem a s hlavičkou `import React from 'react'`. Tyto transpiluje pomocí knihovny Babel a výsledný skript vloží do html template a výsledek pošle jako html response.

## Základní knihovny

/*
!npm install @babel/standalone
!npm install @babel/preset-react
!npm install react
!npm install expressnpm install express
*/

Knihovny uvedené výše importujte v konzoli v adresáři Home

Část tohoto notebooku tvoří zdrojový kód stránky, která je poskytována severem `expressjs`. Zdrojový kód je psán ve verzi jazyka, který nemusí být podporován prohlížečem. Proto se provádí transpilace pomocí knihovny Babel.

In [1]:
import * as Babel from '@babel/standalone'

console.log(Object.keys(Babel.availablePresets))

[
  'env',
  'es2015',
  'es2016',
  'es2017',
  'react',
  'stage-0',
  'stage-1',
  'stage-2',
  'stage-3',
  'es2015-loose',
  'es2015-no-commonjs',
  'typescript',
  'flow'
]


Všimněte si, jak vypadá zdrojový kód po transpilaci.

In [2]:
import * as Babel from '@babel/standalone'

const output = Babel.transform(`
function gen(props) { 
    return (<div>A</div>) 
}
`, 
    { presets: [Babel.availablePresets["react"], Babel.availablePresets["es2015"], ] });
console.log(output.code)

"use strict";

function gen(props) {
  return /*#__PURE__*/React.createElement("div", null, "A");
}


Jako součást html kódu je "napevno" renderování hlavní komponenty `App` do `root`. Příslušný kód je transpilován níže.

In [3]:
import * as Babel from '@babel/standalone'

const output = Babel.transform(`
ReactDOM.render(
      <App />,
      document.getElementById('root')
    );
`, 
    { presets: [Babel.availablePresets["react"], Babel.availablePresets["es2015"], ] });
console.log(output.code)

"use strict";

ReactDOM.render( /*#__PURE__*/React.createElement(App, null), document.getElementById('root'));


Ukázka, jak komplexní může být výsledek transpilace.

In [4]:
import * as Babel from '@babel/standalone'
const output = Babel.transform("function* gen(props) { yield 2 }", 
    { presets: [Babel.availablePresets["react"], Babel.availablePresets["es2015"], ] });

console.log(output.code)

"use strict";

function _typeof(obj) { "@babel/helpers - typeof"; return _typeof = "function" == typeof Symbol && "symbol" == typeof Symbol.iterator ? function (obj) { return typeof obj; } : function (obj) { return obj && "function" == typeof Symbol && obj.constructor === Symbol && obj !== Symbol.prototype ? "symbol" : typeof obj; }, _typeof(obj); }

function _instanceof(left, right) { if (right != null && typeof Symbol !== "undefined" && right[Symbol.hasInstance]) { return !!right[Symbol.hasInstance](left); } else { return left instanceof right; } }

function _regeneratorRuntime() { "use strict"; /*! regenerator-runtime -- Copyright (c) 2014-present, Facebook, Inc. -- license (MIT): https://github.com/facebook/regenerator/blob/main/LICENSE */ _regeneratorRuntime = function _regeneratorRuntime() { return exports; }; var exports = {}, Op = Object.prototype, hasOwn = Op.hasOwnProperty, $Symbol = "function" == typeof Symbol ? Symbol : {}, iteratorSymbol = $Symbol.iterator || "@@iterator",

## Server

### Html template

Níže je definován htmlTemplate, který je založen na Bootstrap knihovně a obsahuje importy pro podporu aplikace s využitím knihovny React.

Jako extra import je proveden import bootstrap ikon.

In [5]:
const htmlTemplate = `
<!DOCTYPE html>
<html lang="en">
<head>
  <title>Bootstrap 5 Example</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>

  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
  <script src="https://unpkg.com/react@17/umd/react.development.js"></script>
  <script src="https://unpkg.com/react-dom@17/umd/react-dom.development.js"></script>

  <script src="https://unpkg.com/@reduxjs/toolkit@1.8.2/dist/redux-toolkit.modern.js"></script>

  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.8.3/font/bootstrap-icons.css">
</head>
<body>

<div class="container-fluid mt-5">
    <div id="root"></div>
</div>


</body>

<script>
###HERE
</script>

<script>
    ReactDOM.render( /*#__PURE__*/React.createElement(App, null), document.getElementById('root'));
</script>
`

Funkce pro získání html template z definovaného notebooku (souboru). Využívá skutečnosti, že notebook je json file.

In [7]:
import fs from 'fs'

const findHtmlTemplate = (fileName) => {
    //nacte data ze souboru
    const rawData = fs.readFileSync(fileName, 'utf8')
    
    //z dat udela dictionary
    const jsonData = JSON.parse(rawData)
    
    //filtrovani tech casti, ktere jsou oznaceny jako code
    const justCodeCells = jsonData.cells.filter(cell => cell.cell_type === 'code')
    
    //filtrovani tech bunek, kde je definovan html template
    const justHtmlTemplateCells = justCodeCells.filter(cell => ((cell.source.length > 0) && (cell.source[0]) && (cell.source[0].startsWith('const htmlTemplate'))))
    
    //vybrani pouze te casti datove struktur (bunky), ktera obsahuje zrdojovy kod
    const rows = justHtmlTemplateCells.map(cell => cell.source)

    //pokud je v souboru vice template, vybrani posledniho
    const lastTemplate = rows[rows.length - 1]
    
    //ignorovani prvniho a posledniho radku kodu
    const htmlTemplate = lastTemplate.slice(1, -1)
    
    //vraceni jedinneho retezce
    return htmlTemplate.join('')
}

console.log(findHtmlTemplate('./05A_express.ipynb'))

<!DOCTYPE html>
<html lang="en">
<head>
  <title>Bootstrap 5 Example</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>

  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
  <script src="https://unpkg.com/react@17/umd/react.development.js"></script>
  <script src="https://unpkg.com/react-dom@17/umd/react-dom.development.js"></script>

  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.8.3/font/bootstrap-icons.css">
</head>
<body>

<div class="container-fluid mt-5">
    <div id="root"></div>
</div>


</body>

<script>
###HERE
</script>

<script>
    ReactDOM.render( 

### Konverzní funkce

Konverzní funkce převede definovaný soubor, který je notebookem do jediného souvislého řetězce. V úvahu jsou brány jen buňky, které jsou zdrojovým kódem a mají na prvním řádku uvedený import knihovny React. Tento řádek je vynechán.

In [9]:
import fs from 'fs'

const convertToJs = (fileName) => {
    const rawData = fs.readFileSync(fileName, 'utf8')
    const jsonData = JSON.parse(rawData)
    const justCodeCells = jsonData.cells.filter(cell => cell.cell_type === 'code')
    const justReactCells = justCodeCells.filter(cell => ((cell.source.length > 0) && (cell.source[0]) && (cell.source[0].startsWith('import React'))))
    const rows = justReactCells.map(cell => cell.source).map(rowGroup => rowGroup.slice(1, rowGroup.length))
    let allLines = rows.flat().join('')
    return allLines
}

const sourceCode = convertToJs('./05A_express.ipynb')
console.log(sourceCode)


const Card = (props) => <div className='card'>{props.children}</div>
const CardHeader = (props) => <div className='card-header'>{props.children}</div>
const CardBody = (props) => <div className='card-body'>{props.children}</div>
const App = (props) => (
    <Card>
        <CardHeader>Greetings</CardHeader>
        <CardBody>Hello world</CardBody>
    </Card>)


### Konverzní funkce s transpilací

Prohlížeče nepodporují všechny aspekty jazyka Javascript. Provádí se tedy překlad z novějšího Javascriptu do staršího Javascriptu, tedy transpilace. Knihovan Babel je knihovnou, která toto umožňuje a je velmi široce využívanou.

In [10]:
import * as Babel from '@babel/standalone'
const babeledSourceCode = (fileName) => Babel.transform(convertToJs(fileName), 
    { presets: [Babel.availablePresets["react"], Babel.availablePresets["es2015"], ] }).code;

console.log(babeledSourceCode('./05A_express.ipynb'))

"use strict";

var Card = function Card(props) {
  return /*#__PURE__*/React.createElement("div", {
    className: "card"
  }, props.children);
};

var CardHeader = function CardHeader(props) {
  return /*#__PURE__*/React.createElement("div", {
    className: "card-header"
  }, props.children);
};

var CardBody = function CardBody(props) {
  return /*#__PURE__*/React.createElement("div", {
    className: "card-body"
  }, props.children);
};

var App = function App(props) {
  return /*#__PURE__*/React.createElement(Card, null, /*#__PURE__*/React.createElement(CardHeader, null, "Greetings"), /*#__PURE__*/React.createElement(CardBody, null, "Hello world"));
};


### Implementace serveru a jeho spuštění

Implementace umožňuje rozložení aplikace do více notebooků.

In [27]:
const express = require('express')
const app = express()
const port = 9992

let servers = {}
const startServer = (port, stop) => {
    const startIt = (typeof stop !== 'undefined') ? stop : true

    if (servers[port]) {
        const result = servers[port].close(() => {
            console.log('Server closed')
        });
        servers[port] = 0
    } 
    
    if (startIt) {
        app.get('/:file', (req, res) => {
            if (req.params.file === 'favicon.ico') {
            res.send('')}
            
            const fileName = './' + req.params.file + '.ipynb'
            const htmlResponse = htmlTemplate.replace('###HERE', babeledSourceCode(fileName))
            res.send(htmlResponse)
        })

        app.get('/', (req, res) => {
            const htmlResponse = htmlTemplate.replace('###HERE', babeledSourceCode('./05A_express.ipynb'))
            res.send(htmlResponse)
        })

        servers[port] = app.listen(port, () => {
          console.log(`Server listening on port ${port}`)
        })
    } 
}
startServer(9992)

Server listening on port 9992


### Ukončení serveru

In [28]:
startServer(9992, false)

Server closed


unknown msg_type: comm_open
unknown msg_type: comm_msg


## Zdrojový kód komponent

V další části jsou uvedeny prvky - komponenty definované pomocí jazyk JSX. Tyto komponenty definují vzhled výsledné html stránky.

Změnou v definici, uložením změny a obnovením html stránky dojde k aktualizaci vizuální reprezentace aplikace.
Nutný kód je integrován do html, tím dochází k obcházení mechanismů cache.

Neopomeňte server spustit, chcete-li provádět experimenty!

### Komponenty pro `Card`

In [104]:
import React from 'react'

const Card = (props) => <div className='card'>{props.children}</div>
const CardHeader = (props) => <div className='card-header'>{props.children}</div>
const CardBody = (props) => <div className='card-body'>{props.children}</div>

### Hlavní komponenta `App`

In [113]:
import React from 'react'

const App = (props) => (
    <Card>
        <CardHeader>Greetings</CardHeader>
        <CardBody>Hello world</CardBody>
    </Card>)